In [1]:
import sys
print(sys.path)

from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.utilities import SQLDatabase
from langchain import PromptTemplate, LLMChain

import json
from datetime import datetime
import pandas as pd
import time
import os
import sys
import json
import ast
import sqlite3


['/home/llmuser', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/llmuser/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']


In [2]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [3]:
local_llm = LlamaCpp(
    model_path="models/mixtral-8x7b-v0.1.Q5_K_M.gguf",
    temperature=0.0,
    n_gpu_layers=-1,
    max_tokens=600,
    n_ctx=10000,
    top_p=1,
    callback_manager=callback_manager,
    stop = ["Q:"],
    verbose=True,  # Verbose is required to pass to the callback manager
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 6 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3060 Ti, compute capability 8.6, VMM: yes
  Device 1: NVIDIA GeForce RTX 3060 Ti, compute capability 8.6, VMM: yes
  Device 2: NVIDIA GeForce RTX 2060, compute capability 7.5, VMM: yes
  Device 3: NVIDIA GeForce RTX 2060, compute capability 7.5, VMM: yes
  Device 4: NVIDIA GeForce RTX 2060, compute capability 7.5, VMM: yes
  Device 5: NVIDIA GeForce RTX 2060, compute capability 7.5, VMM: yes
llama_model_loader: loaded meta data with 25 key-value pairs and 995 tensors from models/mixtral-8x7b-v0.1.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_m

In [4]:


#----------------------------------------------------prompts-----------------------------------------------
schema_linking_prompt = '''Table advisor, columns = [*,s_ID,i_ID]
Table classroom, columns = [*,building,room_number,capacity]
Table course, columns = [*,course_id,title,dept_name,credits]
Table department, columns = [*,dept_name,building,budget]
Table instructor, columns = [*,ID,name,dept_name,salary]
Table prereq, columns = [*,course_id,prereq_id]
Table section, columns = [*,course_id,sec_id,semester,year,building,room_number,time_slot_id]
Table student, columns = [*,ID,name,dept_name,tot_cred]
Table takes, columns = [*,ID,course_id,sec_id,semester,year,grade]
Table teaches, columns = [*,ID,course_id,sec_id,semester,year]
Table time_slot, columns = [*,time_slot_id,day,start_hr,start_min,end_hr,end_min]
Foreign_keys = [course.dept_name = department.dept_name,instructor.dept_name = department.dept_name,section.building = classroom.building,section.room_number = classroom.room_number,section.course_id = course.course_id,teaches.ID = instructor.ID,teaches.course_id = section.course_id,teaches.sec_id = section.sec_id,teaches.semester = section.semester,teaches.year = section.year,student.dept_name = department.dept_name,takes.ID = student.ID,takes.course_id = section.course_id,takes.sec_id = section.sec_id,takes.semester = section.semester,takes.year = section.year,advisor.s_ID = student.ID,advisor.i_ID = instructor.ID,prereq.prereq_id = course.course_id,prereq.course_id = course.course_id]
Q: "Find the buildings which have rooms with capacity more than 50."
A: Let’s think step by step. In the question "Find the buildings which have rooms with capacity more than 50.", we are asked:
"the buildings which have rooms" so we need column = [classroom.capacity]
"rooms with capacity" so we need column = [classroom.building]
Based on the columns and tables, we need these Foreign_keys = [].
Based on the tables, columns, and Foreign_keys, The set of possible cell values are = [50]. So the Schema_links are:
Schema_links: [classroom.building,classroom.capacity,50]

Table department, columns = [*,Department_ID,Name,Creation,Ranking,Budget_in_Billions,Num_Employees]
Table head, columns = [*,head_ID,name,born_state,age]
Table management, columns = [*,department_ID,head_ID,temporary_acting]
Foreign_keys = [management.head_ID = head.head_ID,management.department_ID = department.Department_ID]
Q: "How many heads of the departments are older than 56 ?"
A: Let’s think step by step. In the question "How many heads of the departments are older than 56 ?", we are asked:
"How many heads of the departments" so we need column = [head.*]
"older" so we need column = [head.age]
Based on the columns and tables, we need these Foreign_keys = [].
Based on the tables, columns, and Foreign_keys, The set of possible cell values are = [56]. So the Schema_links are:
Schema_links: [head.*,head.age,56]

Table department, columns = [*,Department_ID,Name,Creation,Ranking,Budget_in_Billions,Num_Employees]
Table head, columns = [*,head_ID,name,born_state,age]
Table management, columns = [*,department_ID,head_ID,temporary_acting]
Foreign_keys = [management.head_ID = head.head_ID,management.department_ID = department.Department_ID]
Q: "what are the distinct creation years of the departments managed by a secretary born in state 'Alabama'?"
A: Let’s think step by step. In the question "what are the distinct creation years of the departments managed by a secretary born in state 'Alabama'?", we are asked:
"distinct creation years of the departments" so we need column = [department.Creation]
"departments managed by" so we need column = [management.department_ID]
"born in" so we need column = [head.born_state]
Based on the columns and tables, we need these Foreign_keys = [department.Department_ID = management.department_ID,management.head_ID = head.head_ID].
Based on the tables, columns, and Foreign_keys, The set of possible cell values are = ['Alabama']. So the Schema_links are:
Schema_links: [department.Creation,department.Department_ID = management.department_ID,head.head_ID = management.head_ID,head.born_state,'Alabama']

'''
classification_prompt = '''Q: "Find the buildings which have rooms with capacity more than 50."
schema_links: [classroom.building,classroom.capacity,50]
A: Let’s think step by step. The SQL query for the question "Find the buildings which have rooms with capacity more than 50." needs these tables = [classroom], so we don't need JOIN.
Plus, it doesn't require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""].
So, we don't need JOIN and don't need nested queries, then the the SQL query can be classified as "EASY".
Label: "EASY"

Q: "What are the names of all instructors who advise students in the math depart sorted by total credits of the student."
schema_links: [advisor.i_id = instructor.id,advisor.s_id = student.id,instructor.name,student.dept_name,student.tot_cred,math]
A: Let’s think step by step. The SQL query for the question "What are the names of all instructors who advise students in the math depart sorted by total credits of the student." needs these tables = [advisor,instructor,student], so we need JOIN.
Plus, it doesn't need nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""].
So, we need JOIN and don't need nested queries, then the the SQL query can be classified as "NON-NESTED".
Label: "NON-NESTED"

Q: "Find the room number of the rooms which can sit 50 to 100 students and their buildings."
schema_links: [classroom.building,classroom.room_number,classroom.capacity,50,100]
A: Let’s think step by step. The SQL query for the question "Find the room number of the rooms which can sit 50 to 100 students and their buildings." needs these tables = [classroom], so we don't need JOIN.
Plus, it doesn't require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""].
So, we don't need JOIN and don't need nested queries, then the the SQL query can be classified as "EASY".
Label: "EASY"

Q: "How many courses that do not have prerequisite?"
schema_links: [course.*,course.course_id = prereq.course_id]
A: Let’s think step by step. The SQL query for the question "How many courses that do not have prerequisite?" needs these tables = [course,prereq], so we need JOIN.
Plus, it requires nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = ["Which courses have prerequisite?"].
So, we need JOIN and need nested queries, then the the SQL query can be classified as "NESTED".
Label: "NESTED"

Q: "Find the title of course that is provided by both Statistics and Psychology departments."
schema_links: [course.title,course.dept_name,Statistics,Psychology]
A: Let’s think step by step. The SQL query for the question "Find the title of course that is provided by both Statistics and Psychology departments." needs these tables = [course], so we don't need JOIN.
Plus, it requires nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = ["Find the titles of courses that is provided by Psychology departments"].
So, we don't need JOIN and need nested queries, then the the SQL query can be classified as "NESTED".
Label: "NESTED"

Q: "What is the name of the instructor who advises the student with the greatest number of total credits?"
schema_links: [advisor.i_id = instructor.id,advisor.s_id = student.id,instructor.name,student.tot_cred ]
A: Let’s think step by step. The SQL query for the question "What is the name of the instructor who advises the student with the greatest number of total credits?" needs these tables = [advisor,instructor,student], so we need JOIN.
Plus, it doesn't need nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""].
So, we need JOIN and don't need nested queries, then the the SQL query can be classified as "NON-NESTED".
Label: "NON-NESTED"

'''

easy_prompt = '''Q: "Find the buildings which have rooms with capacity more than 50."
Schema_links: [classroom.building,classroom.capacity,50]
SQL: SELECT DISTINCT building FROM classroom WHERE capacity  >  50

Q: "Find the room number of the rooms which can sit 50 to 100 students and their buildings."
Schema_links: [classroom.building,classroom.room_number,classroom.capacity,50,100]
SQL: SELECT building ,  room_number FROM classroom WHERE capacity BETWEEN 50 AND 100

Q: "Give the name of the student in the History department with the most credits."
Schema_links: [student.name,student.dept_name,student.tot_cred,History]
SQL: SELECT name FROM student WHERE dept_name  =  'History' ORDER BY tot_cred DESC LIMIT 1


'''
medium_prompt = '''Q: "Find the total budgets of the Marketing or Finance department."
Schema_links: [department.budget,department.dept_name,Marketing,Finance]
A: Let’s think step by step. For creating the SQL for the given question, we need to join these tables = []. First, create an intermediate representation, then use it to construct the SQL query.
Intermediate_representation: select sum(department.budget) from department  where  department.dept_name = \"Marketing\"  or  department.dept_name = \"Finance\"
SQL: SELECT sum(budget) FROM department WHERE dept_name  =  'Marketing' OR dept_name  =  'Finance'

Q: "Find the name and building of the department with the highest budget."
Schema_links: [department.budget,department.dept_name,department.building]
A: Let’s think step by step. For creating the SQL for the given question, we need to join these tables = []. First, create an intermediate representation, then use it to construct the SQL query.
Intermediate_representation: select department.dept_name , department.building from department  order by department.budget desc limit 1
SQL: SELECT dept_name ,  building FROM department ORDER BY budget DESC LIMIT 1

Q: "What is the name and building of the departments whose budget is more than the average budget?"
Schema_links: [department.budget,department.dept_name,department.building]
A: Let’s think step by step. For creating the SQL for the given question, we need to join these tables = []. First, create an intermediate representation, then use it to construct the SQL query.
Intermediate_representation:  select department.dept_name , department.building from department  where  @.@ > avg ( department.budget ) 
SQL: SELECT dept_name ,  building FROM department WHERE budget  >  (SELECT avg(budget) FROM department)

'''
hard_prompt = '''Q: "Find the title of courses that have two prerequisites?"
Schema_links: [course.title,course.course_id = prereq.course_id]
A: Let's think step by step. "Find the title of courses that have two prerequisites?" can be solved by knowing the answer to the following sub-question "What are the titles for courses with two prerequisites?".
The SQL query for the sub-question "What are the titles for courses with two prerequisites?" is SELECT T1.title FROM course AS T1 JOIN prereq AS T2 ON T1.course_id  =  T2.course_id GROUP BY T2.course_id HAVING count(*)  =  2
So, the answer to the question "Find the title of courses that have two prerequisites?" is =
Intermediate_representation: select course.title from course  where  count ( prereq.* )  = 2  group by prereq.course_id
SQL: SELECT T1.title FROM course AS T1 JOIN prereq AS T2 ON T1.course_id  =  T2.course_id GROUP BY T2.course_id HAVING count(*)  =  2

Q: "Find the name and building of the department with the highest budget."
Schema_links: [department.dept_name,department.building,department.budget]
A: Let's think step by step. "Find the name and building of the department with the highest budget." can be solved by knowing the answer to the following sub-question "What is the department name and corresponding building for the department with the greatest budget?".
The SQL query for the sub-question "What is the department name and corresponding building for the department with the greatest budget?" is SELECT dept_name ,  building FROM department ORDER BY budget DESC LIMIT 1
So, the answer to the question "Find the name and building of the department with the highest budget." is =
Intermediate_representation: select department.dept_name , department.building from department  order by department.budget desc limit 1
SQL: SELECT dept_name ,  building FROM department ORDER BY budget DESC LIMIT 1

Q: "Find the title, credit, and department name of courses that have more than one prerequisites?"
Schema_links: [course.title,course.credits,course.dept_name,course.course_id = prereq.course_id]
A: Let's think step by step. "Find the title, credit, and department name of courses that have more than one prerequisites?" can be solved by knowing the answer to the following sub-question "What is the title, credit value, and department name for courses with more than one prerequisite?".
The SQL query for the sub-question "What is the title, credit value, and department name for courses with more than one prerequisite?" is SELECT T1.title ,  T1.credits , T1.dept_name FROM course AS T1 JOIN prereq AS T2 ON T1.course_id  =  T2.course_id GROUP BY T2.course_id HAVING count(*)  >  1
So, the answer to the question "Find the name and building of the department with the highest budget." is =
Intermediate_representation: select course.title , course.credits , course.dept_name from course  where  count ( prereq.* )  > 1  group by prereq.course_id 
SQL: SELECT T1.title ,  T1.credits , T1.dept_name FROM course AS T1 JOIN prereq AS T2 ON T1.course_id  =  T2.course_id GROUP BY T2.course_id HAVING count(*)  >  1

'''
#----------------------------------------------------------------------------------------------------------


In [6]:

DATASET_SCHEMA = '/home/llmuser/models/datasets/spider-2/tables.json'
DATASET = '/home/llmuser/models/datasets/spider-2/dev.json'
OUTPUT_FILE = '/home/llmuser/models/results/DIN-SQL-results'


OUTPUT_DIR = '/home/llmuser/models/results/'
SPIDER_DB_DIR = '/home/llmuser/models/datasets/spider-2/database/' #the path were all databases exist

def load_data(DATASET):
    return pd.read_json(DATASET)

def hard_prompt_maker(test_sample_text,database,schema_links,sub_questions):
  instruction = "# Use the intermediate representation and the schema links to generate the SQL queries for each of the questions.\n"
  fields = find_fields_MYSQL_like("college_2")
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like("college_2") + '\n'
  fields += find_fields_MYSQL_like(database)
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  stepping = f'''\nA: Let's think step by step. "{test_sample_text}" can be solved by knowing the answer to the following sub-question "{sub_questions}".'''
  fields += "\n"
  prompt = instruction +fields + hard_prompt + 'Q: "' + test_sample_text + '"' + '\nschema_links: ' + schema_links + stepping +'\nThe SQL query for the sub-question"'
  return prompt
def medium_prompt_maker(test_sample_text,database,schema_links):
  instruction = "# Use the the schema links and Intermediate_representation to generate the SQL queries for each of the questions.\n"
  fields = find_fields_MYSQL_like("college_2")
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like("college_2") + '\n'
  fields += find_fields_MYSQL_like(database)
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  fields += "\n"
  prompt = instruction +fields + medium_prompt + 'Q: "' + test_sample_text + '\nSchema_links: ' + schema_links + '\nA: Let’s think step by step.'
  return prompt
def easy_prompt_maker(test_sample_text,database,schema_links):
  instruction = "# Use the the schema links to generate the SQL queries for each of the questions.\n"
  fields = find_fields_MYSQL_like("college_2")
  fields += find_fields_MYSQL_like(database)
  fields += "\n"
  prompt = instruction +fields + easy_prompt + 'Q: "' + test_sample_text + '\nSchema_links: ' + schema_links + '\nSQL:'
  return prompt
def classification_prompt_maker(test_sample_text,database,schema_links):
  instruction = "# For the given question, classify it as EASY, NON-NESTED, or NESTED based on nested queries and JOIN.\n"
  instruction += "\nif need nested queries: predict NESTED\n"
  instruction += "elif need JOIN and don't need nested queries: predict NON-NESTED\n"
  instruction += "elif don't need JOIN and don't need nested queries: predict EASY\n\n"
  fields = find_fields_MYSQL_like("college_2")
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like("college_2") + '\n'
  fields += find_fields_MYSQL_like(database)
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  fields += "\n"
  prompt = instruction + fields + classification_prompt + 'Q: "' + test_sample_text + '\nschema_links: ' + schema_links + '\nA: Let’s think step by step.'
  return prompt
def schema_linking_prompt_maker(test_sample_text,database):
  instruction = "# Find the schema_links for generating SQL queries for each question based on the database schema and Foreign keys.\n"
  fields = find_fields_MYSQL_like(database)
  foreign_keys = "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  prompt = instruction + schema_linking_prompt + fields +foreign_keys+ 'Q: "' + test_sample_text + """"\nA: Let’s think step by step."""
  return prompt
def find_foreign_keys_MYSQL_like(db_name):
  df = spider_foreign[spider_foreign['Database name'] == db_name]
  output = "["
  for index, row in df.iterrows():
    output += row['First Table Name'] + '.' + row['First Table Foreign Key'] + " = " + row['Second Table Name'] + '.' + row['Second Table Foreign Key'] + ','
  output= output[:-1] + "]"
  return output
def find_fields_MYSQL_like(db_name):
  df = spider_schema[spider_schema['Database name'] == db_name]
  df = df.groupby(' Table Name')
  output = ""
  for name, group in df:
    output += "Table " +name+ ', columns = ['
    for index, row in group.iterrows():
      output += row[" Field Name"]+','
    output = output[:-1]
    output += "]\n"
  return output
def find_primary_keys_MYSQL_like(db_name):
  df = spider_primary[spider_primary['Database name'] == db_name]
  output = "["
  for index, row in df.iterrows():
    output += row['Table Name'] + '.' + row['Primary Key'] +','
  output = output[:-1]
  output += "]\n"
  return output
def creatiing_schema(DATASET_JSON):
    schema_df = pd.read_json(DATASET_JSON)
    schema_df = schema_df.drop(['column_names','table_names'], axis=1)
    schema = []
    f_keys = []
    p_keys = []
    for index, row in schema_df.iterrows():
        tables = row['table_names_original']
        col_names = row['column_names_original']
        col_types = row['column_types']
        foreign_keys = row['foreign_keys']
        primary_keys = row['primary_keys']
        for col, col_type in zip(col_names, col_types):
            index, col_name = col
            if index == -1:
                for table in tables:
                    schema.append([row['db_id'], table, '*', 'text'])
            else:
                schema.append([row['db_id'], tables[index], col_name, col_type])
        for primary_key in primary_keys:
            index, column = col_names[primary_key]
            p_keys.append([row['db_id'], tables[index], column])
        for foreign_key in foreign_keys:
            first, second = foreign_key
            first_index, first_column = col_names[first]
            second_index, second_column = col_names[second]
            f_keys.append([row['db_id'], tables[first_index], tables[second_index], first_column, second_column])
    spider_schema = pd.DataFrame(schema, columns=['Database name', ' Table Name', ' Field Name', ' Type'])
    spider_primary = pd.DataFrame(p_keys, columns=['Database name', 'Table Name', 'Primary Key'])
    spider_foreign = pd.DataFrame(f_keys,
                        columns=['Database name', 'First Table Name', 'Second Table Name', 'First Table Foreign Key',
                                 'Second Table Foreign Key'])
    return spider_schema,spider_primary,spider_foreign
def debuger(test_sample_text,database,sql):
  instruction = """#### For the given question, use the provided tables, columns, foreign keys, and primary keys to fix the given SQLite SQL QUERY for any issues. If there are any problems, fix them. If there are no issues, return the SQLite SQL QUERY as is.
#### Use the following instructions for fixing the SQL QUERY:
1) Use the database values that are explicitly mentioned in the question.
2) Pay attention to the columns that are used for the JOIN by using the Foreign_keys.
3) Use DESC and DISTINCT when needed.
4) Pay attention to the columns that are used for the GROUP BY statement.
5) Pay attention to the columns that are used for the SELECT statement.
6) Only change the GROUP BY clause when necessary (Avoid redundant columns in GROUP BY).
7) Use GROUP BY on one column only.

"""
  fields = find_fields_MYSQL_like(database)
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  fields += "Primary_keys = " + find_primary_keys_MYSQL_like(database)
  prompt = instruction + fields+ '#### Question: ' + test_sample_text + '\n#### SQLite SQL QUERY\n' + sql +'\n#### SQLite FIXED SQL QUERY\nSELECT'
  return prompt

def local_generation(prompt):
    local_llm.stop = ["Q:"]
    local_llm.max_tokens=600
    return local_llm(prompt)    

def local_debug(prompt):
    local_llm.stop = ["#", ";","\n\n"]
    local_llm.max_tokens=350
    return local_llm(prompt)    


def GPT_generation(prompt):
  response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}],
    n = 1,
    stream = False,
    temperature=0.0,
    max_tokens=600,
    top_p = 1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop = ["Q:"]
  )
  return response['choices'][0]['message']['content']

def GPT4_debug(prompt):
  response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}],
    n = 1,
    stream = False,
    temperature=0.0,
    max_tokens=350,
    top_p = 1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop = ["#", ";","\n\n"]
  )
  return response['choices'][0]['message']['content']

def saveDataJson(OUTPUT_PATH,list):
    # Save the object to a JSON file
    with open(OUTPUT_PATH, 'w') as json_file:
        json.dump(list, json_file)




def load_data_json(DATASET):
    # Open the JSON file and load data
    with open(DATASET, 'r') as json_file:
        return json.load(json_file)

def getdbPath(db_id):
    # Modificar el PATH para donde este la base de datos
    db_path = SPIDER_DB_DIR
    db_uri = db_path + db_id + '/' + db_id + '.sqlite'
    return db_uri

In [ ]:
if __name__ == '__main__':
    spider_schema,spider_primary,spider_foreign = creatiing_schema(DATASET_SCHEMA)
    val_df = load_data(DATASET)
    print(f"Number of data samples {val_df.shape[0]}")
    CODEX = []
    for index, row in val_df.iloc[300:].iterrows():
        #if index < 405: continue #for testing
        print(f"index is {index}")
        print('\nGold: '+row['query'])
        print('\nQuestion:'+row['question'])
        
        SQL = None
        SQL_debug = None
        predicted_class = None
        schema_links = None
        error_type = None
        error_message = None
        error_encountered = False
        
        if not error_encountered:
            schema_links = []
            try:
                schema_links = local_generation(
                    schema_linking_prompt_maker(row['question'], row['db_id']))
            except Exception as e:
                error_encountered = True
                error_type = 'Cannot_generate_schema_link'
                error_message = str(e)                
            try:
                schema_links = schema_links.split("Schema_links: ")[1]
            except Exception as e:
                error_encountered = True
                error_type = 'Cannot_split_schema_link'
                error_message = str(e) 
                schema_links = "[]"
        print('\nschema link: ' + schema_links)
        if not error_encountered:
            classification = None
            try:
                classification = local_generation(
                    classification_prompt_maker(row['question'], row['db_id'], schema_links[1:]))
            except Exception as e:
                error_encountered = True
                error_type = 'Cannot_generate_classification'
                error_message = str(e) 
            try:
                predicted_class = classification.split("Label: ")[1]
            except:
                print("Slicing error for the classification module")
                predicted_class = '"NESTED"'
            print('\nclassification: ' + predicted_class)
        if not error_encountered:
            if '"EASY"' in predicted_class and not error_encountered:
                print("EASY")
                SQL = None
                try:
                    SQL = local_generation(easy_prompt_maker(row['question'], row['db_id'], schema_links))
                except Exception as e:
                    error_encountered = True
                    error_type = 'cannot_generate_easy_query'
                    error_message = str(e) 
            elif '"NON-NESTED"' in predicted_class:
                print("\nNON-NESTED")
                SQL = None
                try:
                    SQL = local_generation(medium_prompt_maker(row['question'], row['db_id'], schema_links))
                except Exception as e:
                    error_encountered = True
                    error_type = 'cannot_generate_medium_query'
                    error_message = str(e) 
                try:
                    SQL = SQL.split("SQL: ")[1]
                except Exception as e:
                    error_encountered = True
                    error_type = 'SQL_slicing_error'
                    error_message = str(e) 
                    SQL = "SELECT"
            else:
                print("\nNESTED")
                SQL = None
                if not error_encountered:
                    try:
                        sub_questions = classification.split('questions = ["')[1].split('"]')[0]
                        SQL = local_generation(
                            hard_prompt_maker(row['question'], row['db_id'], schema_links, sub_questions))
                    except Exception as e:
                        error_encountered = True
                        error_type = 'SQL_slicing_error_or_generation_error'
                        error_message = str(e) 
                try:
                    SQL = SQL.split("SQL: ")[1]
                except Exception as e:
                    error_encountered = True
                    error_type = 'SQL_slicing_error'
                    error_message = str(e) 
                    SQL = "SELECT"
            print('\nSQL:' + SQL)
        debugged_SQL = None
        if not error_encountered:
            try:
                debugged_SQL = local_debug(debuger(row['question'], row['db_id'], SQL)).replace("\n", " ")
            except Exception as e:
                error_encountered = True
                error_type = 'cannot_generate_debug'
                error_message = str(e) 
            SQL_debug = "SELECT " + debugged_SQL
            print('\nSQL debug:'+SQL_debug)

        CODEX.append({'question':row['question'],'gold_query':row['query'],'db_id': row['db_id'], 
                      'predicted_query': SQL, 'predicted_query_debug': SQL_debug, 'predicted_class':predicted_class,
                      'predicted_schema_links':schema_links,
                      'error_encountered':error_encountered, 'error_type': error_type, 'error_message':error_message})
        if index == 600 :  break

    OUTPUT_DIR = '/home/llmuser/models/results/DIN_SQL_Mixtral-300-500.json'
    saveDataJson(OUTPUT_DIR, CODEX)
    

Number of data samples 1034
index is 300

Gold: SELECT document_id ,  document_name ,  document_description FROM Documents

Question:What are the ids, names, and descriptions for all documents?


Llama.generate: prefix-match hit


In [58]:
def queryDB(database, SQL_query):
    conn = sqlite3.connect(database)
    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    # Create a table
    cursor.execute("PRAGMA case_sensitive_like = OFF")
    cursor.execute(SQL_query)
    results = cursor.fetchall()
    return results
    # Close the cursor and connection
    cursor.close()
    conn.close()

def evaluatePredict(CODEX):
    count = 0
    for row in CODEX:
        predict_query_valid = True
        db = getdbPath(row['db_id'])
        error_encountered = False
        error = None
        error_type = None
        predict_success = None
        goldQueryValue = None
        predQueryValue = None 
        try:
            goldQueryValue =  queryDB(db, row['gold_query'])
        except Exception as e:
            error_encountered = True
            error_type = 'Cannot_Query_Gold'
            error_message = str(e)
            predict_query_valid = False
            goldQueryValue = None
        pred_query = row['predicted_query']
        try:
            predQueryValue = queryDB(db, pred_query)
        except Exception as e:
            error_encountered = True
            error_type = 'Cannot_Query_Pred'
            error_message = str(e)
            predQueryValue = None
        if goldQueryValue == predQueryValue :
            predict_success = True
        else:
            predict_success = False
            error_encountered = True
            error_type = 'Gold_Predict_Missmatch'
            error_message = None
        row.update({'predict_success':predict_success, 'gold_result':goldQueryValue, 'predict_result': predQueryValue,'predict_query_valid':predict_query_valid})
        if error_encountered:
            row['error_encountered'] = error_encountered
            row['error_type'] = error_type
            row['error_message'] = error_message
    return CODEX

In [59]:
DATASET1 = load_data_json('/home/llmuser/models/results/DIN_SQL_Mixtral-1-25.json')
DATASET2 = load_data_json('/home/llmuser/models/results/DIN_SQL_Mixtral-25-70.json')
DATASET3 = load_data_json('/home/llmuser/models/results/DIN_SQL_Mixtral-70-300.json')
DATASET4 = load_data_json('/home/llmuser/models/results/DIN_SQL_Mixtral-300-500.json')
CODEX = DATASET1 + DATASET2 + DATASET3 + DATASET4

CODEX_OUT = evaluatePredict(CODEX)

In [69]:
OUTPUT_DIR = '/home/llmuser/models/results/DIN_SQL_Mixtral-500-results.json'
saveDataJson(OUTPUT_DIR, CODEX_OUT)

In [60]:
def calculateAccuracy(DATASET1):
    df = pd.DataFrame(DATASET1)
    success = df[df['predict_success']==True]
    return len(success)/len(df)

accuracy = calculateAccuracy(CODEX_OUT)
accuracy

0.27037773359840955

In [68]:
df = pd.DataFrame(CODEX_OUT)
df = df[df['predict_success']==False]
df = df[['gold_query','predicted_query']]
df

,gold_query,predicted_query
5,"SELECT avg(age) , min(age) , max(age) FROM s...","SELECT AVG(Age) AS AverageAge , MIN(Age) AS M..."
6,"SELECT song_name , song_release_year FROM sin...","SELECT name , release_year FROM song WHERE r..."
7,"SELECT song_name , song_release_year FROM sin...","SELECT name , release_year FROM song WHERE s..."
9,SELECT DISTINCT country FROM singer WHERE age ...,SELECT DISTINCT Country FROM singer WHERE Age...
10,"SELECT country , count(*) FROM singer GROUP B...","SELECT country , COUNT(*) AS 'Number Of Sing..."
...,...,...
497,SELECT avg(injured) FROM death,SELECT AVG(injured) FROM death WHERE killed ...
498,"SELECT T1.killed , T1.injured FROM death AS T...",SELECT
500,"SELECT DISTINCT T1.id , T1.name FROM battle A...","SELECT id , name FROM battle WHERE id IN (SE..."
501,"SELECT T1.id , T1.name FROM battle AS T1 JOIN...",SELECT
